# Portfolio Optimization Using Python

We will build two functions: 
1. Mean variance optimization
2. Min variance portfolio

### Setup

In [1]:
# importing all relevant packages and modules
import numpy as np

### Generating the data set

In [2]:
# Risk free rate
rf_rate = 0.03/12 # div by 12, given we are using monthly returns for the calculations

In [3]:
# Creating an expected return matrix with shape (1,4)
expected_return = np.array([[0.13, 0.06, 0.15, 0.07]])

In [4]:
# Creating a covariance matrix with shape (4,4)
covariance_matrix = np.array([[0.053, 0.009, 0.040, 0.004],
                              [0.009, 0.017, 0.004, 0.009],
                              [0.040, 0.004, 0.084, 0.010],
                              [0.004, 0.009, 0.010, 0.032]
                             ])

### Mean variance optimizer

In [5]:
def mean_variance_optimizer(returns, rf_rate, cov):
    
    """
    returns should be a numpy array with shape (1, number of stocks)
    rf_rate should be a float
    cov should be a numpy array with shape (number of stocks, number of stocks)
    """
    
    # Setup
    from scipy.optimize import minimize, Bounds
    
    # Creating a starting portfolio weight with n_col = 1 and n_row = number of stocks
    wt = np.ones((returns.shape[1],1))*(1.0/returns.shape[1])
    
    # Defining the objective function for the minimization optimization 
    def obj_func_mean_variance(wt, returns, rf_rate, cov):
        wt_ret = returns@wt - rf_rate
        std_dev = (wt.T@cov@wt)**0.5
        sharpe_ratio = wt_ret/std_dev
        return -sharpe_ratio
    
    # Setting variable bounds between 0 and 1 i.e. the optimizer doesnt account for shorting or leveraging
    var_bounds = Bounds(0,1)
    
    # Setting sum of portfolio weights to 1
    var_constraints = {'type':'eq',
                       'fun': lambda wt: 1.0 - np.sum(wt)
                      }
    
    # calling the minizime function from the scipy package
    mean_variance = minimize(obj_func_mean_variance,
                             wt,
                             args=(returns, rf_rate, cov),
                             bounds=var_bounds,
                             constraints=var_constraints
                            )
    
    optimal_sharpe_ratio = -mean_variance.fun
    optimal_port_wt = np.array([mean_variance.x])
    optimal_expected_return = optimal_port_wt@returns.T
    optimal_std_dev = (optimal_port_wt@cov@optimal_port_wt.T)**0.5
    
    print('The optimal weights of the portfolio are:')
    print(['{:.2%}'.format(i) for i in optimal_port_wt[0]])
    print('Optimal expected return is {:.2%}'.format(optimal_expected_return[0,0]))
    print('Optimal std dev is {:.2f}'.format(optimal_std_dev[0,0]))
    print('Optimal sharpe ratio is {:.2f}'.format(optimal_sharpe_ratio[0]))
    
    return optimal_port_wt, optimal_expected_return, optimal_std_dev, optimal_sharpe_ratio

### Minimum variance optimization

In [6]:
def min_variance_optimizer(returns, rf_rate, cov):
    
    """
    returns should be a numpy array with shape (1, number of stocks)
    rf_rate should be a float
    cov should be a numpy array with shape (number of stocks, number of stocks)
    """
    
    # Setup
    from scipy.optimize import minimize, Bounds
    
    # Creating a starting portfolio weight with n_col = 1 and n_row = number of stocks
    wt = np.ones((returns.shape[1],1))*(1.0/returns.shape[1])
    
    # Defining the objective function for the minimization optimization 
    def obj_func_min_var(wt,cov):
        return (wt.T@cov@wt)**0.5
    
    # Setting variable bounds between 0 and 1 i.e. the optimizer doesnt account for shorting or leveraging
    var_bounds = Bounds(0,1)
    
    # Setting sum of portfolio weights to 1
    var_constraints = {'type':'eq',
                       'fun': lambda wt: 1.0 - np.sum(wt)
                      }
    
    # calling the minizime function from the scipy package
    min_var = minimize(obj_func_min_var,
                             wt,
                             args=(cov),
                             bounds=var_bounds,
                             constraints=var_constraints
                            )
    
    min_var_port_wt = np.array([min_var.x])
    min_var_expected_return = min_var_port_wt@returns.T
    min_var_std_dev = min_var.fun
    min_var_sharpe_ratio = (min_var_expected_return-rf_rate)/min_var_std_dev
    
    print('The optimal weights of the portfolio are:')
    print(['{:.2%}'.format(i) for i in min_var_port_wt[0]])
    print('Optimal expected return is {:.2%}'.format(min_var_expected_return[0,0]))
    print('Optimal std dev is {:.2f}'.format(min_var_std_dev))
    print('Optimal sharpe ratio is {:.2f}'.format(min_var_sharpe_ratio[0,0]))
    
    return min_var_port_wt,min_var_expected_return,min_var_std_dev,min_var_sharpe_ratio

### Running the optimization algorithm

In [7]:
mean_var_wt, mean_var_ret, mean_var_sd, mean_var_sr = mean_variance_optimizer(expected_return,rf_rate,covariance_matrix)

The optimal weights of the portfolio are:
['25.26%', '35.79%', '17.21%', '21.74%']
Optimal expected return is 9.53%
Optimal std dev is 0.13
Optimal sharpe ratio is 0.70


In [8]:
min_var_wt, min_var_ret, min_var_sd, min_var_sr = min_variance_optimizer(expected_return,rf_rate,covariance_matrix)

The optimal weights of the portfolio are:
['8.47%', '62.90%', '6.61%', '22.02%']
Optimal expected return is 7.41%
Optimal std dev is 0.12
Optimal sharpe ratio is 0.61


### Testing the functions on data imported from stock returns

In [9]:
import pandas as pd

In [10]:
vangaurd_df = pd.read_csv('vangaurd_funds.csv')

In [11]:
vangaurd_df.isna().sum()

caldt    0
VBINX    0
VPACX    0
VEURX    0
VEIEX    0
VFINX    0
VEXMX    0
VVIAX    0
NAESX    0
VISVX    0
VISGX    0
VIMSX    0
dtype: int64

In [12]:
vangaurd_df.drop(['caldt'],axis=1, inplace=True)

In [13]:
vangaurd_returns = np.array([vangaurd_df.mean(axis=0).to_numpy()])

In [14]:
vangaurd_cov = vangaurd_df.cov().to_numpy()

In [15]:
mean_var_wt, mean_var_ret, mean_var_sd, mean_var_sr = mean_variance_optimizer(vangaurd_returns,rf_rate,vangaurd_cov)

The optimal weights of the portfolio are:
['0.00%', '0.00%', '0.00%', '0.00%', '100.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%']
Optimal expected return is 1.10%
Optimal std dev is 0.04
Optimal sharpe ratio is 0.20


In [16]:
min_var_wt, min_var_ret, min_var_sd, min_var_sr = min_variance_optimizer(vangaurd_returns,rf_rate,vangaurd_cov)

The optimal weights of the portfolio are:
['100.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%']
Optimal expected return is 0.73%
Optimal std dev is 0.03
Optimal sharpe ratio is 0.18


In [35]:
mean_var_wt.ndim?